[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pugapatricia/gestion-documentaria-para-pymes/blob/main/etiquetado/UnionEtiquetado_open.ipynb)

[![Ver en GitHub](https://img.shields.io/badge/GitHub-Repo-black?logo=github)](https://github.com/pugapatricia/gestion-documentaria-para-pymes/tree/main/etiquetado)

#Importaciones

In [84]:
!pip install -q requests msal python-docx PyPDF2 pandas openpyxl python-pptx openai unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.1 MB/s eta 0:00:00


In [85]:
import requests
import msal
from docx import Document
import PyPDF2
import pandas as pd
from pptx import Presentation
import os
import spacy
import torch
from openai import OpenAI
import re
import getpass
import io, os, re, requests
import pandas as pd
import PyPDF2, openpyxl
from pptx import Presentation
from docx import Document
import unidecode

# Configuración

In [140]:
api_key = getpass.getpass("Introduce tu OpenAI API Key: ")
client = OpenAI(api_key=api_key)

Introduce tu OpenAI API Key: ··········


In [141]:
CLIENT_ID = "e3f2393e-7348-47d1-9c64-8d8efe6a5e95"
AUTHORITY = "https://login.microsoftonline.com/consumers"
SCOPE = ["User.Read", "Files.ReadWrite.All"]
ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"


# Conección con OneDrive

In [142]:
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
if "user_code" not in flow:
    raise Exception("No se pudo iniciar el device flow. Revisa tu configuración en Azure.")

print(flow["message"])  # 👉 Copia el código en https://microsoft.com/devicelogin
result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception(f"Error autenticación: {result.get('error_description')}")

access_token = result["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}

# Llamada a la API con tu token de acceso
resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    raise Exception(f"Error al obtener archivos: {resp.text}")
data = resp.json()


To sign in, use a web browser to open the page https://www.microsoft.com/link and enter the code T2KWS8HE to authenticate.


# Funciones

Procesamos cada documento de acuerdo con su tipo (PDF, Word, Excel, etc.) para convertirlo en datos estructurados. Este proceso se realiza mediante las siguientes funciones.

In [143]:
ext_permitidas = {"pdf", "docx", "xlsx", "xls", "csv", "txt", "pptx"}

def leer_pdf(file_path_or_bytes):
    text = ""
    if isinstance(file_path_or_bytes, bytes):
        reader = PyPDF2.PdfReader(io.BytesIO(file_path_or_bytes))
    else:
        reader = PyPDF2.PdfReader(file_path_or_bytes)
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

def leer_docx(file_path_or_bytes):
    if isinstance(file_path_or_bytes, bytes):
        doc = Document(io.BytesIO(file_path_or_bytes))
    else:
        doc = Document(file_path_or_bytes)
    return "\n".join([p.text for p in doc.paragraphs])

def leer_excel(file_path_or_bytes):
    texto = []

    # Detectar extensión si es archivo en disco
    ext = None
    if isinstance(file_path_or_bytes, str):
        ext = file_path_or_bytes.split(".")[-1].lower()
    elif isinstance(file_path_or_bytes, bytes):
        # Por simplicidad asumimos .xlsx si no hay extensión
        ext = "xlsx"

    if ext == "xls":
        # Usar xlrd para archivos antiguos
        temp_file = "temp.xls"
        with open(temp_file, "wb") as f:
            if isinstance(file_path_or_bytes, bytes):
                f.write(file_path_or_bytes)
            else:
                with open(file_path_or_bytes, "rb") as orig:
                    f.write(orig.read())
        wb = xlrd.open_workbook(temp_file)
        for sheet in wb.sheets():
            texto.append(f"\n--- Hoja: {sheet.name} ---\n")
            for row_idx in range(sheet.nrows):
                row = sheet.row_values(row_idx)
                row_text = " ".join([str(c) for c in row if c])
                if row_text.strip():
                    texto.append(row_text)
        os.remove(temp_file)

    else:
        # Usar openpyxl para .xlsx
        if isinstance(file_path_or_bytes, bytes):
            wb = openpyxl.load_workbook(io.BytesIO(file_path_or_bytes), data_only=True, read_only=True)
        else:
            wb = openpyxl.load_workbook(file_path_or_bytes, data_only=True, read_only=True)
        for sheet in wb.worksheets:
            texto.append(f"\n--- Hoja: {sheet.title} ---\n")
            for row in sheet.iter_rows(values_only=True):
                row_text = " ".join([str(c) for c in row if c is not None])
                if row_text.strip():
                    texto.append(row_text)

    return "\n".join(texto)

def leer_csv(file_path_or_bytes):
    if isinstance(file_path_or_bytes, bytes):
        df = pd.read_csv(io.BytesIO(file_path_or_bytes))
    else:
        df = pd.read_csv(file_path_or_bytes)
    return df.to_string()

def leer_txt(file_path_or_bytes):
    if isinstance(file_path_or_bytes, bytes):
        return file_path_or_bytes.decode("utf-8", errors="ignore")
    with open(file_path_or_bytes, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

def leer_pptx(file_path_or_bytes):
    texto = []
    if isinstance(file_path_or_bytes, bytes):
        prs = Presentation(io.BytesIO(file_path_or_bytes))
    else:
        prs = Presentation(file_path_or_bytes)
    for i, slide in enumerate(prs.slides, 1):
        texto.append(f"\n--- Diapositiva {i} ---\n")
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                texto.append(shape.text)
    return "\n".join(texto)

def leer_archivo(item):
    nombre = item.get("name")
    ext = nombre.split(".")[-1].lower()
    if ext not in ext_permitidas:
        return ""

    tmp_path = f"/tmp/{nombre}"
    try:
        # Descargar archivo si no tenemos los bytes
        if "contenido" in item:
            contenido = item["contenido"]
            if isinstance(contenido, bytes):
                with open(tmp_path, "wb") as f:
                    f.write(contenido)
        else:
            r = requests.get(item["@microsoft.graph.downloadUrl"], stream=True)
            r.raise_for_status()
            with open(tmp_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)

        # Leer según extensión
        if ext == "pdf":
            return leer_pdf(tmp_path)
        elif ext == "docx":
            return leer_docx(tmp_path)
        elif ext in {"xlsx", "xls"}:
            return leer_excel(tmp_path)
        elif ext == "pptx":
            return leer_pptx(tmp_path)
        elif ext == "txt":
            return leer_txt(tmp_path)
        elif ext == "csv":
            return leer_csv(tmp_path)
    except requests.RequestException as e:
        print(f"Error descargando {nombre}: {e}")
    except Exception as e:
        print(f"Error procesando {nombre}: {e}")
    finally:
        if os.path.exists(tmp_path):
            os.remove(tmp_path)

    return ""

Función para solicitar a OpenAI la generación de etiquetas/tickers por documento, con un límite máximo

In [144]:
etiquetas_global = set()

def sugerir_tickers(texto, max_etiquetas=10):
    if not texto.strip():
        return []

    prompt = f"""
Eres un asistente que recibe un texto de un documento.
Devuelve solo las {max_etiquetas} palabras más importantes
que podrían usarse como etiquetas del documento, una sola palabra cada una,
en una lista separada por comas. No agregues explicaciones, solo las palabras.

Texto:
{texto}
"""
    try:
        respuesta = client.chat.completions.create(
            model="gpt-5-mini",
            messages=[{"role": "user", "content": prompt}],
        )

        etiquetas = respuesta.choices[0].message.content
        etiquetas_lista = [e.strip().lower() for e in re.split(r'[,\n;]+', etiquetas) if e.strip()]
        etiquetas_lista = list(dict.fromkeys(etiquetas_lista))[:max_etiquetas]
        etiquetas_global.update(etiquetas_lista)

        return etiquetas_lista

    except Exception as e:
        print(f"⚠️ Error al generar etiquetas: {e}")
        return []


Descarga un archivo temporal desde una URL, verifica que su extensión esté permitida y lo lee según su tipo, devolviendo su contenido como texto.

# LISTAR Y PROCESAR ARCHIVOS

In [ ]:
etiquetas = set()  # convertir a set

for item in data.get("value", []):
    if "folder" in item:
        continue

    texto = leer_archivo(item)
    if texto.strip():
        try:
            sugerencias = sugerir_tickers(texto, max_etiquetas=10)
            etiquetas.update([s.strip() for s in sugerencias if s.strip()])
        except Exception as e:
            print(f"⚠️ Error generando etiquetas para {item.get('name')}: {e}")

print("\nTickers finales:\n", sorted(etiquetas))


In [ ]:
etiquetas = list(etiquetas)
etiquetas

In [120]:
len(etiquetas)

156

# ETIQUETADO

In [136]:
def limpiar_texto(texto: str) -> str:
    if not texto:
        return ""
    texto = re.sub(r'\d+\n', '', texto)
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'[^a-zA-Záéíóúüñ0-9\s,.\-()/:]', '', texto)
    return texto.strip()

def etiquetar_texto(texto, etiquetas, max_etiquetas=5, modelo="gpt-5-mini"):
    etiquetas_norm = {unidecode.unidecode(e.lower().replace(" ", "")): e for e in etiquetas}

    prompt = f"""
Analiza cuidadosamente el siguiente texto y selecciona hasta {max_etiquetas} etiquetas que mejor representen sus temas principales.
Usa únicamente las etiquetas de la lista exacta que se proporciona más abajo, no inventes ninguna etiqueta.
Lista de etiquetas: {', '.join(etiquetas)}

Texto: \"\"\"{texto}\"\"\"

Devuelve únicamente las etiquetas seleccionadas, separadas por comas, sin explicaciones ni palabras adicionales.
Responde de forma precisa, considerando todo el contenido del texto.
"""

    respuesta = client.chat.completions.create(
        model=modelo,
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=400
    )

    raw = re.split(r'[,\n;]+', respuesta.choices[0].message.content)
    resultado = []
    for e in raw:
        key = unidecode.unidecode(e.strip().lower().replace(" ", ""))
        if key in etiquetas_norm:
            resultado.append(etiquetas_norm[key])
        elif key.endswith("s") and key[:-1] in etiquetas_norm:
            resultado.append(etiquetas_norm[key[:-1]])

    return resultado[:max_etiquetas]

In [137]:
texto = """El contrato de arrendamiento establece claramente la renta, el plazo, las obligaciones del arrendador y arrendatario,
y las cláusulas de prórroga y fianza. Además, se menciona la propiedad intelectual relacionada con los proyectos
de divulgación científica, literatura y poesía, incluyendo derechos de autor y acuerdos de confidencialidad (NDA).
El inmueble objeto del contrato deberá cumplir con las leyes y la jurisdicción aplicable, respetando las normas de
financiación y las resoluciones de los organismos competentes. También se contemplan actividades culturales
como teatro y arte, así como cursos monográficos de astrofísica y biodiversidad.
"""
texto_limpio = limpiar_texto(texto)

resultado = etiquetar_texto(texto_limpio, etiquetas, max_etiquetas=3)
print("Etiquetas detectadas:", resultado)


Etiquetas detectadas: []


In [138]:
archivos_permitidos = [
    item for item in data.get("value", [])
    if "folder" not in item and item["name"].split(".")[-1].lower() in ext_permitidas
]

archivos_a_procesar = archivos_permitidos[:3]

if archivos_a_procesar:
    resultados = {}

    for item in archivos_a_procesar:
        nombre = item["name"]
        download_url = item["@microsoft.graph.downloadUrl"]

        print(f"Procesando archivo: {nombre}")

        file_bytes = requests.get(download_url).content
        texto = leer_archivo(item)
        texto = limpiar_texto(texto)

        if texto:
            etiquetas_detectadas = etiquetar_texto(texto,etiquetas)
        else:
            etiquetas_detectadas = []

        resultados[nombre] = etiquetas_detectadas
        print(f"{nombre} → {etiquetas_detectadas}")


Procesando archivo: 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf
Error descargando 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf: 401 Client Error: Unauthorized for url: https://my.microsoftpersonalcontent.com/personal/9f68595d59f0b70b/_layouts/15/download.aspx?UniqueId=c1bb0c8d-9040-4e3f-8ae9-2438b23c4747&Translate=false&tempauth=v1e.eyJzaXRlaWQiOiJjMjFhZmM2OC0xNDUxLTQyODEtOTNjYy0yOGZhNDQ4M2Y4YmIiLCJhcHBfZGlzcGxheW5hbWUiOiJFdGlxdWV0YXMiLCJhcHBpZCI6ImUzZjIzOTNlLTczNDgtNDdkMS05YzY0LThkOGVmZTZhNWU5NSIsImF1ZCI6IjAwMDAwMDAzLTAwMDAtMGZmMS1jZTAwLTAwMDAwMDAwMDAwMC9teS5taWNyb3NvZnRwZXJzb25hbGNvbnRlbnQuY29tQDkxODgwNDBkLTZjNjctNGM1Yi1iMTEyLTM2YTMwNGI2NmRhZCIsImV4cCI6IjE3NjEwODU0NjUifQ.9sjufW-qidW4TD3KItOnzuH7R-27udgacNMAkFlgUifxAf-KIFhLb5LQm8gfFhsYnCqO7uhWr8u3jR8FlpD1F6CnycCUw6E2YcR0oXK-kqFH2vuh9yaVw63ka3fArZzDyViZH4QRu6S0JludlTTbhjUENdo4njyBkktm3_eBa8RmeiGmBxavmbFyzHdmmhLIgul4wn-2zmRDq0VL-rnmRxtqFUe75d-37B-UJFVDDcBjS8RS1QPCfoxgA4W6mT8WE